# Библиотеки и CUDA

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch

In [3]:
from transformers import AutoProcessor, AutoModelForTextToWaveform

In [4]:
from transformers import VitsTokenizer, VitsModel, set_seed

In [5]:
print(torch.cuda.is_available())

False


In [6]:
import numpy as np

In [7]:
from datasets import load_dataset

In [8]:
import soundfile as sf

In [9]:
from IPython.display import Audio # штука для работы с аудио

In [78]:
import ast

# Модуль Б. Разработка модели машинного обучения 

In [10]:
processor = AutoProcessor.from_pretrained('facebook/mms-tts-rus') # хз что за параша; обработка звуковданных??
modell =  VitsModel.from_pretrained('facebook/mms-tts-rus') # моделька

____

In [11]:
tokenizer = VitsTokenizer.from_pretrained('facebook/mms-tts-rus') # токенизатор

данные (цсв в датасеты):

In [13]:
train = load_dataset('csv', data_files = r'..\Module_A\data_selections\train2.csv')
test = load_dataset('csv', data_files = r'..\Module_A\data_selections\test2.csv')
valid = load_dataset('csv', data_files = r'..\Module_A\data_selections\valid2.csv')

In [97]:
print(np.array(train['train']['audio_array']))

['[-3.05175781e-05  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00\n  0.00000000e+00  3.05175781e-05]'
 '[0.         0.         0.         ... 0.00332642 0.00204468 0.00247192]'
 '[-3.05175781e-05 -6.10351562e-05  0.00000000e+00 ...  3.66210938e-04\n -1.83105469e-04 -7.01904297e-04]'
 ...
 '[ 0.00021362  0.00027466  0.00039673 ... -0.00048828 -0.0007019\n -0.00024414]'
 '[-3.05175781e-05 -3.05175781e-05  0.00000000e+00 ... -5.79833984e-04\n  1.64794922e-03  1.73950195e-03]'
 '[0.         0.         0.         ... 0.06008911 0.06115723 0.06155396]']


In [171]:
inputs = tokenizer(text="дисперсия детерминация дискриминация дискретизация", return_tensors="pt")

set_seed(550)  # make deterministic

with torch.no_grad():
    outputs = modell(inputs["input_ids"])
#outputs.waveform.shape
#torch.Size([1, 45824])

In [172]:
print(outputs[0])

tensor([[-5.3776e-05, -3.7270e-05,  3.3108e-05,  ...,  7.0600e-05,
         -1.7303e-05, -9.4949e-05]])


In [173]:
print(np.array(outputs[0][0]))

[-5.3775893e-05 -3.7270129e-05  3.3107717e-05 ...  7.0600239e-05
 -1.7302529e-05 -9.4948839e-05]


In [174]:
Audio(outputs[0][0], rate = 16000)

In [ ]:
#sf.write('aftg1.wav', outputs[0][0], samplerate = 16000)

_____

In [14]:
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

In [40]:
import ast

In [112]:
class TTSDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        
        row = self.dataset[idx]

        text = row["text"]
        audio = row["audio_array"]
        print(audio)
        audio = np.array(audio)

        # токенизация
        tokens = self.tokenizer(text, return_tensors="pt", padding="max_length", truncation=True)
        input_ids = tokens.input_ids.squeeze()
        attention_mask = tokens.attention_mask.squeeze()

        audio_tensor = torch.tensor(ast.literal_eval(audio), dtype=torch.float32)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": audio_tensor
        }

In [113]:
class DataCollatorTTS:
    def __call__(self, batch):
        input_ids = [item["input_ids"] for item in batch]
        attention_mask = [item["attention_mask"] for item in batch]
        labels = [item["labels"] for item in batch]

        return {
            "input_ids": pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id),
            "attention_mask": pad_sequence(attention_mask, batch_first=True, padding_value=0),
            "labels": pad_sequence(labels, batch_first=True)  # padding для аудио
        }

In [114]:
class VitsForTraining(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.model = VitsModel.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        # outputs.waveform: [batch, time]
        # labels: [batch, time]
        if labels is not None:
            loss = torch.nn.functional.mse_loss(outputs.waveform, labels)
            return {"loss": loss, "logits": outputs.waveform}
        return {"logits": outputs.waveform}

In [115]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./tts_finetune",
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    num_train_epochs=10,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    report_to="none"
)


In [117]:
trainer = Trainer(
    model=VitsForTraining("facebook/mms-tts-rus"),
    args=training_args,
    eval_dataset=TTSDataset(valid['train'], tokenizer),
    train_dataset=TTSDataset(train['train'], tokenizer),
    data_collator=DataCollatorTTS()
)

In [118]:
trainer.train()

[0.         0.         0.         ... 0.00042725 0.00036621 0.00039673]


ValueError: malformed node or string: array('[0.         0.         0.         ... 0.00042725 0.00036621 0.00039673]',
      dtype='<U71')